# 01 - Data exploration

### Import packages and load the data

In [4]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import plotly.graph_objects as go

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [5]:
data_input_path = '/Users/szejozsef00/Desktop/MSC/MSC 2. félév/DS Lab I/DSLAB1/data/processed/'

In [6]:
df = pd.read_csv(data_input_path + 'processed_data.csv',sep=';',decimal=',')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df = df.fillna(0)

### Transform the data

In [7]:
# Add Date column
df['DATE'] = pd.to_datetime(df['DATETIME']).dt.date

In [8]:
df['DATE'].max()

datetime.date(2010, 9, 24)

In [9]:
# Melt the dataframe to DATE-DATETIME-LOCATION-VALUE format
melted_fact_df = pd.melt(df, id_vars=['DATE','DATETIME'],var_name='LOCATION', value_name='VALUE')
melted_fact_df['LOCATION'] = melted_fact_df['LOCATION'].astype(int)
lmelted_fact_df = melted_fact_df.sort_values('LOCATION')
melted_fact_df.head(5)

,DATE,DATETIME,LOCATION,VALUE
0,2009-07-02,2009-07-02 00:00:00,0,-79.50
1,2009-07-02,2009-07-02 00:05:00,0,-22.81
2,2009-07-02,2009-07-02 00:10:00,0,23.02
3,2009-07-02,2009-07-02 00:15:00,0,21.36
4,2009-07-02,2009-07-02 00:20:00,0,25.18


In [10]:
melted_fact_df['LOCATION'].nunique()

1916

In [11]:
melted_fact_df = melted_fact_df[melted_fact_df['LOCATION'] < 197]

In [12]:
len(melted_fact_df[(melted_fact_df['DATE'] == datetime.date(2009, 7, 2)) & (melted_fact_df['LOCATION'] == 1)])

288

### Scale the data

In [13]:
#using min-max scaler, scale the data
custom_scaler = pd.DataFrame()

custom_scaler['actual_min'] = melted_fact_df.groupby('LOCATION')['VALUE'].min()
custom_scaler['actual_max'] = melted_fact_df.groupby('LOCATION')['VALUE'].max()

custom_scaler['min'] = custom_scaler[['actual_min']].min(axis=1)
custom_scaler['max'] = custom_scaler[['actual_max']].max(axis=1)

custom_scaler = custom_scaler[['min', 'max']].reset_index()

scaled_df = melted_fact_df.merge(custom_scaler, on='LOCATION', how='left').copy(deep=True)

custom_scaler.to_csv(data_input_path + 'custom_scaler.csv', index=False)

In [ ]:
import pickle
from sklearn.preprocessing import MinMaxScaler

def min_max_scale_by_location(df, value_col='VALUE', location_col='LOCATION'):
    scalers = {}
    df_scaled = df.copy()

    # Iterate over each unique location
    for location in df[location_col].unique():
        # Filter the dataframe for the current location
        location_df = df[df[location_col] == location]
        
        # Create a MinMaxScaler for the current location
        scaler = MinMaxScaler()
        
        # Fit and transform the VALUE column for the current location
        df_scaled.loc[df[location_col] == location, 'VALUE_SCALED'] = scaler.fit_transform(location_df[[value_col]])
        
        # Store the scaler for the current location
        scalers[location] = scaler

    # Save the scaler dictionary to a file
    with open(data_input_path + 'scalers.pkl', 'wb') as f:
        pickle.dump(scalers, f)
    
    return df_scaled

df_scaled = min_max_scale_by_location(melted_fact_df)

df_scaled.head()

,DATE,DATETIME,LOCATION,VALUE,VALUE_SCALED
0,2009-07-02,2009-07-02 00:00:00,0,-79.50000,0.425602
1,2009-07-02,2009-07-02 00:05:00,0,-22.81000,0.463518
2,2009-07-02,2009-07-02 00:10:00,0,23.02000,0.494171
3,2009-07-02,2009-07-02 00:15:00,0,21.36000,0.493061
4,2009-07-02,2009-07-02 00:20:00,0,25.18000,0.495616
...,...,...,...,...,...
25531195,2010-09-24,2010-09-24 23:35:00,196,-143.04000,0.695082
25531196,2010-09-24,2010-09-24 23:40:00,196,-146.16000,0.688431
25531197,2010-09-24,2010-09-24 23:45:00,196,-144.52000,0.691927
25531198,2010-09-24,2010-09-24 23:50:00,196,-143.90000,0.693249


In [15]:
# def inverse_transform_by_location(df_scaled, scalers_path, value_col='VALUE_SCALED', location_col='LOCATION'):
#     # Load the scalers from the pickle file
#     with open(scalers_path, 'rb') as f:
#         scalers = pickle.load(f)
    
#     # Create a copy of the dataframe to avoid modifying the original one
#     df_original = df_scaled.copy()
    
#     # Iterate over each unique location
#     for location in df_scaled[location_col].unique():
#         # Filter the dataframe for the current location
#         location_df = df_scaled[df_scaled[location_col] == location]
        
#         # Get the scaler for the current location
#         scaler = scalers[location]
        
#         # Inverse transform the VALUE_SCALED column for the current location
#         df_original.loc[df_scaled[location_col] == location, 'VALUE_ORIGINAL'] = scaler.inverse_transform(location_df[[value_col]])
    
#     return df_original

# # Example usage
# scalers_path = data_input_path + 'scalers.pkl'
# df_original = inverse_transform_by_location(df_scaled, scalers_path)

# df_original

### Check autocorrelation

In [16]:
def plot_acf_pacf(location_id, data, lags):
    # Filter the data for the given location
    location_data = data[data['LOCATION'] == location_id]['VALUE_SCALED']

    # Plot ACF
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    plot_acf(location_data, ax=plt.gca(), lags=lags)
    plt.title(f'ACF for Location {location_id}')

    # Plot PACF
    plt.subplot(122)
    plot_pacf(location_data, ax=plt.gca(), lags=lags)
    plt.title(f'PACF for Location {location_id}')

    plt.tight_layout()
    plt.show()

In [17]:
# plot_acf_pacf(0, df_scaled,2016)

In [18]:
# Extracting features from DATETIME
df_scaled['YEAR'] = df_scaled['DATETIME'].dt.year
df_scaled['MONTH'] = df_scaled['DATETIME'].dt.month
df_scaled['DAY_OF_WEEK'] = df_scaled['DATETIME'].dt.day_of_week
df_scaled['DAY'] = df_scaled['DATETIME'].dt.day
df_scaled['HOUR'] = df_scaled['DATETIME'].dt.hour

df_scaled.head()

,DATE,DATETIME,LOCATION,VALUE,VALUE_SCALED,YEAR,MONTH,DAY_OF_WEEK,DAY,HOUR
0,2009-07-02,2009-07-02 00:00:00,0,-79.50,0.425602,2009,7,3,2,0
1,2009-07-02,2009-07-02 00:05:00,0,-22.81,0.463518,2009,7,3,2,0
2,2009-07-02,2009-07-02 00:10:00,0,23.02,0.494171,2009,7,3,2,0
3,2009-07-02,2009-07-02 00:15:00,0,21.36,0.493061,2009,7,3,2,0
4,2009-07-02,2009-07-02 00:20:00,0,25.18,0.495616,2009,7,3,2,0


In [19]:
# hourly_summary = df_scaled.groupby(['LOCATION','YEAR','MONTH','DAY','HOUR']).agg({'VALUE_SCALED':'sum'}).reset_index()
# location_0_data = hourly_summary[hourly_summary['LOCATION'] == 0]

# # Run the plot_acf_pacf function for location 0 and lags = 12
# plot_acf_pacf(0, location_0_data, lags=168)

In [20]:
# hourly_summary = df_scaled.groupby(['LOCATION','YEAR','MONTH','DAY','HOUR']).agg({'VALUE_SCALED':'sum'}).reset_index()
# location_0_data = hourly_summary[hourly_summary['LOCATION'] == 0]

# # Run the plot_acf_pacf function for location 0 and lags = 12
# plot_acf_pacf(0, location_0_data, lags=168*2)

In [ ]:
loc_0_df = df_scaled[df_scaled['LOCATION'] == 0]
loc_0_df.head()

,DATE,DATETIME,LOCATION,VALUE,VALUE_SCALED,YEAR,MONTH,DAY_OF_WEEK,DAY,HOUR
0,2009-07-02,2009-07-02 00:00:00,0,-79.50,0.425602,2009,7,3,2,0
1,2009-07-02,2009-07-02 00:05:00,0,-22.81,0.463518,2009,7,3,2,0
2,2009-07-02,2009-07-02 00:10:00,0,23.02,0.494171,2009,7,3,2,0
3,2009-07-02,2009-07-02 00:15:00,0,21.36,0.493061,2009,7,3,2,0
4,2009-07-02,2009-07-02 00:20:00,0,25.18,0.495616,2009,7,3,2,0
...,...,...,...,...,...,...,...,...,...,...
129595,2010-09-24,2010-09-24 23:35:00,0,192.08,0.607245,2010,9,4,24,23
129596,2010-09-24,2010-09-24 23:40:00,0,193.10,0.607927,2010,9,4,24,23
129597,2010-09-24,2010-09-24 23:45:00,0,209.31,0.618769,2010,9,4,24,23
129598,2010-09-24,2010-09-24 23:50:00,0,206.11,0.616629,2010,9,4,24,23


### Add holiday feature

In [22]:
from prophet.make_holidays import make_holidays_df
import re

hu_holidays = make_holidays_df(
    year_list=[2009, 2010],
    country='HU'
)

hu_holidays['holiday'] = hu_holidays.holiday.apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x).strip())

aut_holidays = make_holidays_df(
    year_list=[2009, 2010],
    country='AT'
)

aut_holidays['holiday'] = aut_holidays.holiday.apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x).strip())

holidays = pd.merge(hu_holidays, aut_holidays, how='inner', on=['ds', 'holiday'])
holidays.columns = ['DATE', 'holiday']
holidays['DATE'] = pd.to_datetime(holidays['DATE'])
df_scaled['DATE'] = pd.to_datetime(df_scaled['DATE'])

df_scaled = df_scaled.merge(holidays, on = 'DATE', how = 'left', validate='m:1')

In [23]:
df_scaled.head()

,DATE,DATETIME,LOCATION,VALUE,VALUE_SCALED,YEAR,MONTH,DAY_OF_WEEK,DAY,HOUR,holiday
0,2009-07-02,2009-07-02 00:00:00,0,-79.50000,0.425602,2009,7,3,2,0,NaN
1,2009-07-02,2009-07-02 00:05:00,0,-22.81000,0.463518,2009,7,3,2,0,NaN
2,2009-07-02,2009-07-02 00:10:00,0,23.02000,0.494171,2009,7,3,2,0,NaN
3,2009-07-02,2009-07-02 00:15:00,0,21.36000,0.493061,2009,7,3,2,0,NaN
4,2009-07-02,2009-07-02 00:20:00,0,25.18000,0.495616,2009,7,3,2,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
25531195,2010-09-24,2010-09-24 23:35:00,196,-143.04000,0.695082,2010,9,4,24,23,NaN
25531196,2010-09-24,2010-09-24 23:40:00,196,-146.16000,0.688431,2010,9,4,24,23,NaN
25531197,2010-09-24,2010-09-24 23:45:00,196,-144.52000,0.691927,2010,9,4,24,23,NaN
25531198,2010-09-24,2010-09-24 23:50:00,196,-143.90000,0.693249,2010,9,4,24,23,NaN


### Encode the date base features

In [24]:
#encode the values
df_encoded = pd.concat([
  df_scaled,
  # pd.get_dummies(df_scaled['LOCATION'], dtype=int, prefix='LOC'),
  pd.get_dummies(df_scaled['holiday'], dtype=int),
  pd.get_dummies(df_scaled['DAY'], dtype=int, prefix='day'),
  pd.get_dummies(df_scaled['HOUR'], dtype=int, prefix='hour'),
], axis=1)

In [25]:
# df_encoded.drop(columns=['MONTH','DAY_OF_WEEK','holiday', 'DAY', 'HOUR','YEAR']).to_csv(data_input_path + 'modelling_df.csv',index=False)